# 推薦システムにおける評価手法の話

## 概要

この記事では、推薦システムにおける評価手法について詳述する。評価手法の定義や性質、応用例について数式とPythonのコードを用いて具体例を示す。また、評価手法のメリットとデメリットについても論じ、具体的な利用例として「movielens-100k」データセットを用いた実装例を紹介する。



## ソースコード


### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/rec/gr/06/06_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/rec/gr/06/06_nb.ipynb)


## 実行環境
OSはmacOSである。LinuxやUnixのコマンドとはオプションが異なりますので注意していただきたい。

In [1]:
!sw_vers

ProductName:		macOS
ProductVersion:		13.5.1
BuildVersion:		22G90


In [2]:
!python -V

Python 3.9.17


pandasのテーブルを見やすいようにHTMLのテーブルにCSSの設定を行います。

In [ ]:
from IPython.core.display import HTML

style = """
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

    .dataframe thead th {
        text-align: left;
        padding: 5px;
    }

    .dataframe tbody tr th {
        vertical-align: top;
        padding: 5px;
    }

    .dataframe tbody tr:hover {
        background-color: #ffff99;
    }

    .dataframe {
        background-color: white;
        color: black;
        font-size: 16px;
    }

</style>
"""
HTML(style)

基本的なライブラリをインポートし watermark を利用してそのバージョンを確認しておきます。
ついでに乱数のseedの設定をします。

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
import random

import scipy
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

seed = 123
random_state = 123

random.seed(seed)
np.random.seed(seed)


from watermark import watermark

print(watermark(python=True, watermark=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.9.17
IPython version      : 8.17.2

scipy     : 1.11.2
numpy     : 1.25.2
matplotlib: 3.8.1

Watermark: 2.4.3



## 評価手法の定義と性質

推薦システムの評価手法は、システムがユーザーに対してどれだけ適切な推薦を行っているかを測定する方法である。評価手法は、推薦システムの性能を比較・改善するために不可欠である。

### 主要な評価指標

推薦システムの評価には、以下の主要な指標が使用される。

1. **精度 (Precision)**:
   推薦されたアイテムのうち、ユーザーが実際に好むアイテムの割合。
   $$ \text{Precision} = \frac{|\{推奨アイテム\} \cap \{好まれるアイテム\}|}{|\{推奨アイテム\}|} $$

2. **再現率 (Recall)**:
   ユーザーが好むアイテムのうち、推薦されたアイテムの割合。
   $$ \text{Recall} = \frac{|\{推奨アイテム\} \cap \{好まれるアイテム\}|}{|\{好まれるアイテム\}|} $$

3. **F1スコア (F1 Score)**:
   精度と再現率の調和平均。
   $$ \text{F1 Score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}} $$

4. **平均絶対誤差 (MAE)**:
   推薦システムの予測と実際の評価の絶対値の平均。
   $$ \text{MAE} = \frac{1}{N} \sum_{i=1}^{N} |r_i - \hat{r}_i| $$

5. **二乗平均平方根誤差 (RMSE)**:
   推薦システムの予測と実際の評価の二乗平均平方根。
   $$ \text{RMSE} = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (r_i - \hat{r}_i)^2} $$

## 応用例と実装

次に、これらの評価指標を用いて「movielens-100k」データセットを使った実装例を示す。

### データセットの準備

まず、「movielens-100k」データセットをロードし、評価行列を準備する。



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

# データセットの読み込み
ratings = pd.read_csv("u.data", sep="\t", header=None, names=["user_id", "movie_id", "rating", "timestamp"])

# データセットの分割
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

# 評価行列の作成
R_train = train_data.pivot(index="user_id", columns="movie_id", values="rating").fillna(0)
R_test = test_data.pivot(index="user_id", columns="movie_id", values="rating").fillna(0)

# 評価行列をnumpy配列に変換
R_train = R_train.values
R_test = R_test.values

In [ ]:



### 評価指標の計算

次に、予測評価行列を生成し、評価指標を計算する。ここでは、簡単な平均ベースの予測を使用する。



In [ ]:
# ユーザー平均に基づく予測
user_mean = np.mean(R_train, axis=1)
R_pred = np.tile(user_mean, (R_train.shape[1], 1)).T

# MAEとRMSEの計算
mae = mean_absolute_error(R_test[R_test > 0], R_pred[R_test > 0])
rmse = sqrt(mean_squared_error(R_test[R_test > 0], R_pred[R_test > 0]))

print(f"MAE: {round(mae, 2)}")
print(f"RMSE: {round(rmse, 2)}")

In [ ]:


### 精度、再現率、F1スコアの計算

次に、精度、再現率、F1スコアを計算する。



In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 二値予測: 予測評価が3以上である場合を「好まれる」とする
R_pred_binary = (R_pred >= 3).astype(int)
R_test_binary = (R_test >= 3).astype(int)

# 精度、再現率、F1スコアの計算
precision = precision_score(R_test_binary[R_test > 0], R_pred_binary[R_test > 0])
recall = recall_score(R_test_binary[R_test > 0], R_pred_binary[R_test > 0])
f1 = f1_score(R_test_binary[R_test > 0], R_pred_binary[R_test > 0])

print(f"Precision: {round(precision, 2)}")
print(f"Recall: {round(recall, 2)}")
print(f"F1 Score: {round(f1, 2)}")



## メリットとデメリット

### メリット

- **評価の多様性**: 複数の評価指標を使用することで、システムの性能を多角的に評価できる。
- **改善の指針**: 指標ごとにシステムの強みや弱点が明確になり、改善の方向性が見える。

### デメリット

- **計算コスト**: データが大規模になると、評価指標の計算に時間がかかる。
- **バイアス**: 特定の評価指標に偏ると、システムの総合的な性能が見落とされる可能性がある。

## 結論

この記事では、推薦システムにおける評価手法について詳述した。具体的な定義や数式、Pythonコードを用いた具体例を示し、メリットとデメリットを論じた。評価手法は、システムの性能を多角的に評価し、改善の方向性を見出すために重要である。

### 参考文献

- Wikipedia: [Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall)
- Scikit-learn: [Metrics and scoring](https://scikit-learn.org/stable/modules/model_evaluation.html)